<a href="https://colab.research.google.com/github/woodRock/deep-learning-goodfellow/blob/main/chapter_7_regularization_for_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7 | Regularization for Deep Learning

We defined regularization as "any modification we make to a learning algorithm that is intended to reduce its generalization error but not its training error".

In [8]:
import numpy as np

class Tensor (object):

    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):

        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id

        self.creators = creators
        self.creation_op = creation_op
        self.children = {}

        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True

    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):

            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad

            # grads must not have grads of their own
            assert grad.autograd == False

            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if (self.creators is not None and
               (self.all_children_grads_accounted_for() or
                grad_origin is None)):

                if (self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)

                if (self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if (self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)

                if (self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)

                if (self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if ("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,self.creators[0].data.shape[dim]))

                if ("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))

                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())

                if (self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))

                if (self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))

                if (self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))

                if (self.creation_op == "cross_entropy"):
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))

    @property
    def shape(self):
        return self.data.shape

    def __add__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if (self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)

    def __sub__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)

    def __mul__(self, other):
        if (self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)

    def sum(self, dim):
        if (self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))

    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)

        if (self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)

    def transpose(self):
        if (self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")

        return Tensor(self.data.transpose())

    def mm(self, x):
        if (self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))

    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

    def sigmoid(self):
        if (self.autograd):
            return Tensor(1 / (1 + np.exp(-self.data)), autograd=True, creators=[self], creation_op = "sigmoid")
        return Tensor(1 / (1 + np.exp(-self.data)))

    def tanh(self):
        if (self.autograd):
            return Tensor(np.tanh(self.data), autograd=True, creators=[self], creation_op = "tanh")
        return Tensor(np.tanh(self.data))

    def index_select(self, indices):
        if (self.autograd):
            new = Tensor(self.data[indices.data], autograd=True, creators=[self], creation_op = "index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])

    def cross_entropy(self, target_indices):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp, axis=len(self.data.shape)-1, keepdims=True)
        target_dist = target.data
        loss = -(np.log(softmax_output) * target_dist).sum(axis=1).mean()

        if (self.autograd):
            out = Tensor(loss, autograd=True, creators=[self], creation_op="cross_entropy")
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out

        return Tensor(loss)

class SGD(object):
    def __init__(self, parameters, alpha=0.1):
        self.parameters = parameters
        self.alpha = alpha

    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0

    def step(self, zero=True):
        for p in self.parameters:
            p.data -= p.grad.data * self.alpha

            if (zero):
                p.grad.data *= 0

class Layer(object):
    def __init__(self):
        self.parameters = list()

    def get_parameters(self):
        return self.parameters

class Linear(Layer):
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0 / (n_inputs))
        self.weight = Tensor(W, autograd=True)
        self.bias = Tensor(np.zeros(n_outputs), autograd=True)

        self.parameters.append(self.weight)
        self.parameters.append(self.bias)

    def forward(self, input):
        return input.mm(self.weight) + self.bias.expand(0, len(input.data))

class Sequential(Layer):
    def __init__(self, layers=list(), training=True):
        super().__init__()
        self.layers = layers
        self.training = training

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, input):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                layer.training = self.training
            input = layer.forward(input)
        return input

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    def get_parameters(self):
        params = list()
        for l in self.layers:
            params += l.get_parameters()
        return params

class Dropout(Layer):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p
        self.mask = None

    def forward(self, input):
        # Only apply dropout when training.
        if self.training:
            # Multiply by 1 / (1 - p) to balance out the extra sensitivity.
            self.mask = np.random.binomial(1, 1-self.p, input.shape) / (1-self.p)
            return input * Tensor(self.mask, autograd=input.autograd)
        return input

    def backward(self, grad):
        return grad * self.mask


class MSELoss(Layer):

    def __init__(self):
        super().__init__()

    def forward(self, pred, target):
        return ((pred - target) * (pred - target)).sum(0)


class CrossEntropyLoss(object):
    def __init__(self):
        super().__init__()

    def forward(self, input, target):
        return input.cross_entropy(target)

class Tanh(Layer):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input.tanh()

class Sigmoid(Layer):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input.sigmoid()

class Relu(Layer):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input.relu()


from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train[0:1_000].reshape(1_000,28*28) / 255
X_test = X_test[0:1_000].reshape(1_000,28*28) / 255
y_train = y_train[0:1_000]
y_test = y_test[0:1_000]

# Onehot encoding
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

# Hyperparameters
alpha = 0.001
epochs = 100
batch_size = 100
input_dim = 784
hidden_size = 100
output_dim = 10

# Initialize the model
model = Sequential([
    Linear(input_dim,hidden_size),
    Tanh(),
    Dropout(p=0.5),
    Linear(hidden_size, hidden_size),
    Tanh(),
    Dropout(p=0.5),
    Linear(hidden_size, output_dim),
    Sigmoid()
])

criterion = CrossEntropyLoss()
optimizer = SGD(model.get_parameters(), alpha)

# Training loop
for j in range(epochs):
    correct_cnt = 0
    model.train()
    for i in range(int(len(X_train) / batch_size)):
        batch_start, batch_end = ((i * batch_size), ((i+1) * batch_size))
        input = Tensor(X_train[batch_start:batch_end], autograd=True)
        target = Tensor(y_train[batch_start:batch_end], autograd=True)

        # Forward pass
        prediction = model.forward(input)

        loss = criterion.forward(prediction, target)

        for k, p in enumerate(prediction.data):
            pred_label = p
            true_label = y_train[batch_start+k: batch_start+k+1]
            correct_cnt += int(np.argmax(pred_label.data) == np.argmax(true_label))

        # Back propgation
        loss.backward()
        # Update the weights0
        optimizer.step()

    training_accuracy = correct_cnt / float(len(y_train))

    if (j % 10 == 0 or j == epochs - 1):
        model.eval()
        test_correct_cnt = 0
        for i in range(int(len(X_test) / batch_size)):
          batch_start, batch_end = ((i * batch_size), ((i+1) * batch_size))
          input = Tensor(X_test[batch_start:batch_end], autograd=True)
          target = Tensor(y_test[batch_start:batch_end], autograd=True)

          # Forward pass
          prediction = model.forward(input)

          for k, p in enumerate(prediction.data):
              pred_label = p
              true_label = y_test[batch_start+k: batch_start+k+1]
              test_correct_cnt += int(np.argmax(pred_label.data) == np.argmax(true_label))

        test_accuracy = test_correct_cnt / float(len(y_test))
        print(f"I: {j} \t accuracy: {training_accuracy} \t test accuracy: {test_accuracy}")

I: 0 	 accuracy: 0.123 	 test accuracy: 0.231
I: 10 	 accuracy: 0.625 	 test accuracy: 0.705
I: 20 	 accuracy: 0.727 	 test accuracy: 0.748
I: 30 	 accuracy: 0.752 	 test accuracy: 0.769
I: 40 	 accuracy: 0.779 	 test accuracy: 0.783
I: 50 	 accuracy: 0.809 	 test accuracy: 0.788
I: 60 	 accuracy: 0.797 	 test accuracy: 0.794
I: 70 	 accuracy: 0.826 	 test accuracy: 0.799
I: 80 	 accuracy: 0.845 	 test accuracy: 0.809
I: 90 	 accuracy: 0.864 	 test accuracy: 0.809
I: 99 	 accuracy: 0.874 	 test accuracy: 0.804


 # $ L^2$ Regularization - Ridge regresssion

The $L^2$ parameters norm penalty common known as **weight decay**. This regularization strategy drives the weights closer to the origin, by adding a regularozation term $\Omega (\theta) = \frac{1}{2} || w ||^2_2$ to the objective function. In other academic communities, $L^2$ reguliarzation is also known as **ridge regression** or **Tikhonoz regularization**.

Such a model has the total objective function

$$
  \tilde{J} (w;X,y) = \frac{\alpha}{2} w^\top w + J(w;X,y)
$$

with the corresponding parameter gradient

$$
  \nabla_w \tilde{j} (W;X,y) = \alpha w + \nabla_w J(w;X,y)
$$

To take a single gradient step to update the weights, we perform this update

$$
  w \leftarrow w - \epsilon(\alpha w + \nabla_w J(w;X,y))
$$

Written another way, the update is

$$
  w \leftarrow (1 - \epsilon \alpha)w - \epsilon \nabla_w J(w;X,y)
$$

We can see that the addition of the weight decay term  has modified the learning rule to multiplicatively shrink the weight vector by a constant factor on each step, just before performing the usual gradient update. THis describes what happens in a single step

We will further simplify analysis by making a quadratic approximation to the objective function in the neighborhood of the value of weights tat obtian minimal unregularized training cost $w^* = argmin_w J(w)$. If the objective function is truly quadratic, as in the case of fitting a linear regression model with mean equared error, then the approximation is perfect. The approximation $\hat{J}$ is given by

$$
  \hat{J}(\theta) = J(w^*) + \frac{1}{2}(w - w^*)^\top H(w - w^*)
$$

where $H$ is the Hessian matrix of $J$ with respect to $w$ evaluated at $w^*$. There is no first-order term in the quadratic approximation, becuase $w^*$ is defined to be a minimum, where the gradient vanishes. Likewise, because $w^*$ is the location of a minimum of $J$, we can conclude $H$ is positive semidefinite.

Remember: Hessian is the Jacobian of the gradients. And positive semidefinite means the eigenvalues are zero or positive.

The minimum of $\hat{J}$ occurs where its gradient

$$
  \nabla_w \hat{J} (w) = H (w - w^*)
$$

is equal to $0$.

To studey the effect of weight decay, we modify the equation above by adding the weight decay gradient. We can now solve for the minimum of the regularized versio of $\hat{J}$. We use the variable $\tilde{w}$ to represent the location of the minimum.

$$
  \alpha \tilde{w} + H(\tilde{w} - w^*) = 0 \\
  (H + \alpha I) \tilde{w} = Hw^* \\
  \tilde{w} = (H + \alpha I)^{-1} Hw^*
$$

As $alpha$ appproaches $0$, the regularized solution $\tilde{w}$ approaches $w^*$. But what happens as $\alpha$ grows. Becuase $H$ is real and symmetric, we can decompose it into a diagonal matrix $\Lambda$ and an orthonormal basis of eigenvectors, $Q$, such that $H = Q\Lambda Q^\top$.

Applying the decomposition, we obtain

$$
  \tilde{w} = (Q\Lambda Q^\top + \alpha I)^{-1} Q\Lambda Q^\top w^* \\
  = [Q(\Lambda + \alpha I) Q^\top]^{-1} Q\Lambda Q^\top w^* \\
  = Q(\Lambda + \alpha I)^{-1} \Lambda Q^\top w^*
$$

We see that the effect of the weight decay is to recale $w^*$ along the axes defined by the eigenvectors of $H$. Specifically, the component $w^*$ that is aligned with the $i$th eigenvector of $J$ is recaled by a factor of $\frac{\lambda_i}{\lambda_i + \alpha}$

For linear regression the cost function is the sum of squared errors:

$$
  (Xw - y)^\top (Xw - y)
$$

When we add $L^2$ regularization, the objective function changes to

$$
  (Xw - y)^\top (Xw - y) + \frac{1}{2}\alpha w^\top w
$$

This changes the normal equations for the solution from

$$
  w = (X^\top X)^{-1} X^\top y
$$

to

$$
  w = (X^\top X + \alpha I)^{-1} X^\top y
$$

The matrix $X^\top X$ in the equation above is proportional to the covariance matrix $\frac{1}{m} X^\top X$. Using the $L^2$ regularization replaces this matrix with $(X^\top X + \alpha I)^{-1}$. The new matrix is the same as the original one, but with the addition of $\alpha$ to the diagonal. The diagonal entires of this matrix correspond to the variance of each input feature. We can see that $L^2$ regularization cuases the learning algorithm to percieve the input $X$ as having higher variance, which makes it shrink the weights on features whose covariance with the output target is low compared to this added variance.

In [47]:
class RidgeLoss(Layer):
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = Tensor(alpha, autograd=True)  # Regularization strength

    def forward(self, pred, target, model):
        # Compute the Mean Squared Error (MSE)
        mse_loss = ((pred - target)*(pred - target)).sum(0)

        # Compute the Ridge (L2) regularization term
        ridge_loss = Tensor(0.0, autograd=True)
        for param in model.get_parameters():
            ridge_loss += Tensor((param.data ** 2).sum(), autograd=True)

        # Combine MSE loss and Ridge regularization
        total_loss = mse_loss + self.alpha * ridge_loss
        return total_loss

# Hyperparameters
input_dim = 784
output_dim = 10
hidden_size = 50
alpha = 0.01
epochs = 100
batch_size = 100
lambda_reg = 1

# Initialize the model
model = Sequential([
    Linear(input_dim, hidden_size),
    Tanh(),
    Linear(hidden_size, output_dim),
    Sigmoid()
])

criterion = RidgeLoss(lambda_reg)
optimizer = SGD(model.get_parameters(), alpha)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    correct_cnt = 0
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        input = Tensor(batch_X, autograd=True)
        target = Tensor(batch_y, autograd=True)

        # Forward pass
        prediction = model.forward(input)
        loss = criterion.forward(prediction, target, model)
        total_loss += loss.data

        for i in range(len(prediction.data)):
            pred_label = prediction.data[i]
            true_label = batch_y[i]
            correct_cnt += int(np.argmax(pred_label.data) == np.argmax(true_label))

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

    accuracy = correct_cnt / float(len(y_train))
    avg_loss = total_loss / (len(X_train) / batch_size)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Accuracy: {accuracy}, Average Loss: {avg_loss}")

# Evaluation
model.eval()
X_test_tensor = Tensor(X_test, autograd=True)
y_pred = model.forward(X_test_tensor)
test_accuracy = 0
for i in range(len(y_pred.data)):
    pred_label = y_pred.data[i]
    true_label = y_test [i]
    test_accuracy += int(np.argmax(pred_label.data) == np.argmax(true_label))
test_accuracy = test_accuracy / float(len(y_test))
print(f"Test Accuracy: {test_accuracy}")
mse = ((y_pred.data - y_test) ** 2).mean()
print(f"Test MSE: {mse}")

Epoch 0, Accuracy: 0.218, Average Loss: [151.14980994 151.82856124 152.87026549 152.32658156 153.29569342
 153.71550238 150.40138061 152.9466566  152.21460277 153.98905984]
Epoch 10, Accuracy: 0.92, Average Loss: [199.26083543 199.6837563  200.48314094 200.24990728 199.86259332
 201.50202756 199.75383511 200.09991734 200.85029789 200.68123405]
Epoch 20, Accuracy: 0.964, Average Loss: [237.09703836 237.49574915 237.78261246 237.60957519 237.43434781
 238.15779056 237.34933208 237.49808163 237.80691848 237.94105403]
Epoch 30, Accuracy: 0.979, Average Loss: [264.91860295 265.25474381 265.35436521 265.31848694 265.17820094
 265.5597743  265.07975477 265.13401559 265.29853381 265.59751995]
Epoch 40, Accuracy: 0.984, Average Loss: [286.20442692 286.52582521 286.49997669 286.56695953 286.38973139
 286.69534539 286.34417183 286.33361676 286.4033552  286.75650289]
Epoch 50, Accuracy: 0.985, Average Loss: [303.35812479 303.67111967 303.55752762 303.69041233 303.50361865
 303.69803671 303.4843223

# $L^1$ Regularization - Lasso regression

While $L^2$ weight decay is the most common form of weight decay, there are other ways to penalize the size of model parameters. Another option is to use the $L^1$ regularization, otherwise known as **lasso regression**.

Formally, $L^1$ regularization on the model parameter $w$ is defined as

$$
  \Omega(\theta) = ||w||_1 = \sum_i |w_i|
$$

that is, as the sum of absolute values of the individual parameters.

As with $L^2$ weight decay, $L^1$ weight decay controls the strength of the regularization by scaling the penalty $\Omega$ using a positive hyperparameter $\alpha$. Thus, the regularized objective function $\tilde{J} (w;X,y)$ is given by

$$
  \tilde{J} (w;X,y) = \alpha ||w||_1 + J(w;X,y)
$$

with the corresponding gradient

$$
  \nabla_w \tilde{J} (w;X,y) = \alpha sign(w) + \nabla_w J(X,y;w)
$$

where $sign(w)$ is simply the sign of $w$ applied element-wise.

We can see that the regularization contribution to the gradient no longer scales linearly with each $w_i$; instead it is a constant factor with sign equal to $sign(w_i)$. One consequence of this form of gradient is that we will not necessarily see clean algebraic solutions to quadratic approximiations of $J(X,y;w)$ as we did for $L^2$ regularization.

Our simple linear model has a quadratic cost function that we can represent via its Taylor series. Alternatively, we could imagine that this is a truncated Taylor series approximating the cost function of a more sophisticated model. The gradient in this setting is given by

$$
  \nabla_w \hat{J}(w) = H(w - w^*)
$$

where, again, $H$ is a Hessian matrix of $J$ with respect to $w$ evaluated at $w^*$

Becuase the $L^1$ penalty does not admit clean algebraic expressions in the case of a fully general Hessian, we will also make the further simplifying assumption that the Hessian is diagonal, $H = diag([H_{1,1}, \cdots, H_{n,n}])$ where $H_{i,i} > 0$. This assumption holds if the data for the linear regression problem has been preprocessed to remove all correlation between input features, which is accomplished using PCA.

Our quadratic approximation of the $L^1$ regularizaed objective  function decomposes into a sum over the parameters.

$$
  \hat{J}(w;X,y) - J(w^*;x,y) + \sum_i [\frac{1}{2} H_{i,i}(w_i - w_i^*)^2 + \alpha |w_i|]
$$

The problem of minmizing the cost function has an analytical solution with the following form:

$$
  w_i = sign(w^*) max \{ |w_i| - \frac{\alpha}{H_{i,i}}, 0\}
$$

Consider the situation where $w_i^* > 0$ for all $i$. There are two possible outcomes:

1. The case where $w_i^* \le \frac{\alpha}{H_{i,i}}$. Here the optimal value of $w_i$ under the regularized objective is simply $w_i = 0$. This occurs because the contribution of $J(w;X,y)$ to the regularized objective $\hat{J}(w;X,y)$ is overwhelmed - in direction $i$ - by the $L^1$ regularization, which puhses the value of $w_i$ to zero.
2.  The case where $w_i^* > \frac{\alpha}{H_{i,i}}$. In this case, the regularization does not move the potimal value of $w_i$ to zero but instead just shifts it in the direction by a distance equal to $\frac{\alpha}{H_{i,i}}$

A similar process happens when $w^*_i < 0$, but with the $L^1$ penalty making $w_i$ less negative by $\frac{\alpha}{H_{i,i}}$, or 0.

In comparison to $L^2$ regularization, $L^1$ regularization results in a solution that is more **sparse**. Spartisity in this context refers to the fact that some parameters have an optimial value of zero. The sparsity of $L^1$ regularization is a qualitatively different behaviour than arises with $L^2$ regularization.

This equation gave the solution for $\tilde{w} for $L^2$ reguarlization:

$$
  \tilde{w} = (H + \alpha I)^{-1} Hw^*
$$

If we revisit that equation using the assumption of diagonal and positive semidefinite Hessian $H$ that we introduced for our analysis of $L^1$ regularization, we find that

$$
   \tilde{w} = \frac{H_{i,i}}{H_{i,i} + \alpha} w_i^*
$$

If $w_i$ is nonzero, then $\tilde{w}_i$ remains nonzero. This demonstrates that $L^2$ regularization does not cause the parameters to become spare, while $L^1$ regularization may do so for large enough $\alpha$.

$L^2$ regualirzation is equivalent to MAP Bayesian inference with a Gaussian prior on the weights. For $L^1$ regularization the penalty $ \alpha \Omega (w) = \alpha \sum_i |w_i| $ used to regularize the cost function is equivalent to the log-prior term that is maximized by MAP Bayesian inference when the prior is an isotropic Laplace distribution over $w \in \mathbb{R}^n$:

$$
  log p(w) = \sum_i \log Laplace(w_i;0,\frac{1}{a}) \\
  = -a ||w||_1 + n \log \alpha - n \log 2
$$

From the point of view of learning via maximization with respect to $w$, we can ignore $\log \alpha - \log 2$ terms because they do not depend on $w$.

Remember: An even simpler version is the **isotropic** Gaussian distribution, whose covariance matrix is a scalar times the identity matrix.

In [53]:
class LassoLoss(Layer):
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = Tensor(alpha, autograd=True)  # Regularization strength

    def forward(self, pred, target, model):
        # Compute the Mean Squared Error (MSE)
        mse_loss = ((pred - target) * (pred - target)).sum(0)

        # Compute the Lasso (L1) regularization term
        lasso_loss = Tensor(0.0, autograd=True)
        for param in model.get_parameters():
            lasso_loss += Tensor(np.abs(param.data).sum(), autograd=True)

        # Combine MSE loss and Lasso regularization
        total_loss = mse_loss + self.alpha * lasso_loss
        return total_loss

# Hyperparameters
input_dim = 784
output_dim = 10
hidden_size = 50
alpha = 0.01
epochs = 100
batch_size = 100
lambda_reg = 1

# Initialize the model
model = Sequential([
    Linear(input_dim, hidden_size),
    Tanh(),
    Linear(hidden_size, output_dim),
    Sigmoid()
])

criterion = LassoLoss(lambda_reg)
optimizer = SGD(model.get_parameters(), alpha)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    correct_cnt = 0
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        input = Tensor(batch_X, autograd=True)
        target = Tensor(batch_y, autograd=True)

        # Forward pass
        prediction = model.forward(input)
        loss = criterion.forward(prediction, target, model)
        total_loss += loss.data

        for i in range(len(prediction.data)):
            pred_label = prediction.data[i]
            true_label = batch_y[i]
            correct_cnt += int(np.argmax(pred_label.data) == np.argmax(true_label))

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

    accuracy = correct_cnt / float(len(y_train))
    avg_loss = total_loss / (len(X_train) / batch_size)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Accuracy: {accuracy}, Average Loss: {avg_loss}")

# Evaluation
model.eval()
X_test_tensor = Tensor(X_test, autograd=True)
y_pred = model.forward(X_test_tensor)
test_accuracy = 0
for i in range(len(y_pred.data)):
    pred_label = y_pred.data[i]
    true_label = y_test [i]
    test_accuracy += int(np.argmax(pred_label.data) == np.argmax(true_label))
test_accuracy = test_accuracy / float(len(y_test))
print(f"Test Accuracy: {test_accuracy}")
mse = ((y_pred.data - y_test) ** 2).mean()
print(f"Test MSE: {mse}")

Epoch 0, Accuracy: 0.203, Average Loss: [1803.72702978 1803.43550497 1802.81006316 1801.10295619 1802.14006753
 1801.81631922 1802.28535311 1803.97387147 1801.79079882 1804.52030859]
Epoch 10, Accuracy: 0.913, Average Loss: [1965.37804101 1965.74152099 1966.55385917 1966.27247352 1966.21412502
 1967.47787618 1965.99973786 1966.62955769 1966.97365152 1970.48844513]
Epoch 20, Accuracy: 0.964, Average Loss: [2076.15847128 2076.50152311 2076.83546918 2076.66772604 2076.60086932
 2077.22109548 2076.54075072 2076.79492616 2076.98531746 2077.66164822]
Epoch 30, Accuracy: 0.98, Average Loss: [2152.21874875 2152.52027732 2152.58057775 2152.54336297 2152.51528918
 2152.86425708 2152.49186719 2152.48821143 2152.69272137 2153.13352728]
Epoch 40, Accuracy: 0.985, Average Loss: [2206.14368068 2206.43164528 2206.37951588 2206.41045741 2206.3456064
 2206.51568285 2206.3412459  2206.28079002 2206.4924667  2206.77119811]
Epoch 50, Accuracy: 0.988, Average Loss: [2245.51227023 2245.76145156 2245.67002219